In [2]:
import numpy as np
import scipy as sp

import matplotlib.pyplot as plt
import matplotlib.axes as axe
import pandas as pd
import datetime as dt
import gurobipy as gp
from gurobipy import GRB
import cvxpy as cp
import yaml

import random
from itertools import chain, combinations, tee
import time


# Network Structure

In [2]:
class network_structure():
    def __init__(self, edges_nodes_matrix):
        # Structure of edges_nodes_matrix
        # An array of 2d-arrays, with each 2d-array encoding the start and end node indices of an edge.
        
        assert edges_nodes_matrix.shape[1] == 2, "We must have edges_nodes_matrix.shape[1] == 2"
        
        self.edges_nodes_matrix = edges_nodes_matrix
        self.nodes = np.unique(self.edges_nodes_matrix.flatten())
        self.edges = np.array(range(self.edges_nodes_matrix.shape[0]))
        self.num_nodes = len(self.nodes)
        self.num_edges = len(self.edges)
        
        self.construct_nodes_dict()
        self.construct_edges_dict()
#         self.construct_depth_dicts()
#         self.construct_height_dicts()
    
        origin_nodes = [node for node in self.nodes if self.nodes_dict[node]['incoming_edges'] == []]
        destination_nodes = [node for node in self.nodes if self.nodes_dict[node]['outgoing_edges'] == []]
        
        assert len(origin_nodes) == 1, "For now, we consider only single-origin \
            single-destination networks."
        assert len(destination_nodes) == 1, "For now, we consider only single-origin \
            single-destination networks."
        self.origin_node = origin_nodes[0]
        self.destination_node = destination_nodes[0]
        self.intermediate_nodes = [node for node in self.nodes if node not in [self.origin_node, self.destination_node]]
        
        self.start_edges = self.nodes_dict[self.origin_node]["outgoing_edges"]
        self.end_edges = self.nodes_dict[self.destination_node]["incoming_edges"]
        
        self.construct_routes_dict()
    
        return

    
    # Network structure functions:
    
    def construct_nodes_dict(self):
        self.nodes_dict = {}
        for node in self.nodes:
            self.nodes_dict[node] = {}
            self.nodes_dict[node]['incoming_edges'] = list((self.edges_nodes_matrix[:, 1] == node).nonzero()[0])
            self.nodes_dict[node]['outgoing_edges'] = list((self.edges_nodes_matrix[:, 0] == node).nonzero()[0])
            
            self.nodes_dict[node]['previous_nodes'] = list(set(self.edges_nodes_matrix[edge, 0] for edge in self.nodes_dict[node]['incoming_edges']))
            self.nodes_dict[node]['next_nodes'] = list(set(self.edges_nodes_matrix[edge, 1] for edge in self.nodes_dict[node]['outgoing_edges']))
        
        return

    def construct_edges_dict(self):
        self.edges_dict = {}

        for edge in self.edges:
            self.edges_dict[edge] = {}

        for node in self.nodes:
            if self.nodes_dict[node]['incoming_edges'] != []:
                for edge in self.nodes_dict[node]['incoming_edges']:
                    self.edges_dict[edge]['end_node'] = node

            if self.nodes_dict[node]['outgoing_edges'] != []:
                for edge in self.nodes_dict[node]['outgoing_edges']:
                    self.edges_dict[edge]['start_node'] = node

        for edge in self.edges:
            self.edges_dict[edge] = {}
            start_node = int(np.array([node for node in self.nodes if edge in self.nodes_dict[node]['outgoing_edges']]))
            end_node = int(np.array([node for node in self.nodes if edge in self.nodes_dict[node]['incoming_edges']]))
            self.edges_dict[edge]['start_node'] = start_node
            self.edges_dict[edge]['end_node'] = end_node
            self.edges_dict[edge]['incoming_edges'] = self.nodes_dict[start_node]['incoming_edges']
            self.edges_dict[edge]['outgoing_edges'] = self.nodes_dict[end_node]['outgoing_edges']
        return
    
    def construct_routes_dict(self):
        self.routes = [[edge] for edge in self.start_edges]
        
        route_construction_finished = False
        while route_construction_finished is False:
            route_construction_finished = True
            routes_temp = []
            for route_segment in self.routes:
                
#                 print()
#                 print("route_segment:", route_segment)
#                 print("route_segment[-1]:", route_segment[-1])
#                 print("self.edges_dict[route_segment[-1]]:", self.edges_dict[route_segment[-1]])
#                 print("self.edges_dict[route_segment[-1]][outgoing_edges]:", self.edges_dict[route_segment[-1]]["outgoing_edges"])
# #                 print()
                
                if self.edges_dict[route_segment[-1]]["outgoing_edges"] != []:
                    route_construction_finished = False
                    for edge in self.edges_dict[route_segment[-1]]["outgoing_edges"]:
                        routes_temp.append(route_segment + [edge])
                else:
                    routes_temp.append(route_segment)
                
#                 print("routes_temp:", routes_temp)
                
                
            if route_construction_finished is True:
                break
            self.routes = routes_temp
            
#             print()
#             print("self.routes:", self.routes)
#             print()

        return
    
    def print_nodes_dict(self):
        print()
        print("Printing nodes:")
        for node_key, node_value in self.nodes_dict.items():
            print()
            print("Node", node_key)
            for key, value in node_value.items():
                print(key, ":", value)
        print()
        return
    
    def print_edges_dict(self):
        print()
        print("Printing edges:")
        for edge_key, edge_value in self.edges_dict.items():
            print()
            print("Edge", edge_key)
            for key, value in edge_value.items():
                print(key, ":", value)
        print()
        return
    
    def print_routes(self):
        print()
        print("Printing routes:")
        print()
        for route_index, route_edges in enumerate(self.routes):
            print("Route", route_index, ":", route_edges)
        return
    
    def print_all(self):
        print()
        print("Printing network information:")
        print()
        print("Origin node:", self.origin_node)
        print("Destination node:", self.destination_node)
        print("Number of nodes:", self.num_nodes)
        print()
        print("Starting edges:", self.start_edges)
        print("Ending edges:", self.end_edges)
        print("Number of edges:", self.num_edges)
        
        self.print_nodes_dict()
        self.print_edges_dict()
        self.print_routes()
        
        return
    

In [3]:
# list_1 = [1, 2]
# list_2 = [3, 4, 5]
# list_1 = list_1 + [6]
# list_2.append(list_1)
# list_2

In [4]:
edges_nodes_matrix = np.array([[0, 1], [0, 2], [1, 2], [1, 3], [2, 3]])
netw_example = network_structure(edges_nodes_matrix)

# print("netw_example.edges:", netw_example.edges)
# print()
# print("netw_example.nodes:", netw_example.nodes)
# print()
# print("netw_example.edges_dict:", netw_example.edges_dict)
# print()
# print("netw_example.nodes_dict:", netw_example.nodes_dict)
# print()
# print("netw_example.routes:", netw_example.routes)
# print()

# netw_example.print_nodes_dict()
# netw_example.print_edges_dict()
# netw_example.print_routes()

netw_example.print_all()
# netw_example.intermediate_nodes



Printing network information:

Origin node: 0
Destination node: 3
Number of nodes: 4

Starting edges: [0, 1]
Ending edges: [3, 4]
Number of edges: 5

Printing nodes:

Node 0
incoming_edges : []
outgoing_edges : [0, 1]
previous_nodes : []
next_nodes : [1, 2]

Node 1
incoming_edges : [0]
outgoing_edges : [2, 3]
previous_nodes : [0]
next_nodes : [2, 3]

Node 2
incoming_edges : [1, 2]
outgoing_edges : [4]
previous_nodes : [0, 1]
next_nodes : [3]

Node 3
incoming_edges : [3, 4]
outgoing_edges : []
previous_nodes : [1, 2]
next_nodes : []


Printing edges:

Edge 0
start_node : 0
end_node : 1
incoming_edges : []
outgoing_edges : [2, 3]

Edge 1
start_node : 0
end_node : 2
incoming_edges : []
outgoing_edges : [4]

Edge 2
start_node : 1
end_node : 2
incoming_edges : [0]
outgoing_edges : [4]

Edge 3
start_node : 1
end_node : 3
incoming_edges : [0]
outgoing_edges : []

Edge 4
start_node : 2
end_node : 3
incoming_edges : [1, 2]
outgoing_edges : []


Printing routes:

Route 0 : [0, 2, 4]
Route 1 : [

# General DBCP Equilibrium Solver

## (Special Case) Quartic Polynomial Latency Functions

In [5]:
# grad = np.array([0, 1, 2, 3, 4])
grad = np.array([2, 4, 0, 1, 3])

for id_temp, entry_temp in enumerate(grad):
    print("id_temp, entry_temp:", id_temp, entry_temp)


id_temp, entry_temp: 0 2
id_temp, entry_temp: 1 4
id_temp, entry_temp: 2 0
id_temp, entry_temp: 3 1
id_temp, entry_temp: 4 3


In [6]:
def welfare_obj(network, lambda_E, lambda_R, lambda_I, tau, v_I_array, v_E_array, y_el, y_in, alpha, \
                a_ex = np.array([0.0, 0.0, 0.0, 0.0, 1.0]), \
                a_gp = np.array([0.0, 0.0, 0.0, 0.0, 1.0])):
    
    assert np.all(a_ex >= 0.0), "All entries of a_ex must be non-negative."
    assert np.all(a_gp >= 0.0), "All entries of a_gp must be non-negative."
    assert a_ex.shape == a_gp.shape, "a_ex and a_gp must have the same shape."
    assert len(a_ex.shape) in [1, 2], "a_ex and a_gp must be either a vector or a matrix."
    
    if len(a_ex.shape) == 1:
        latency_params_length = a_ex.shape[0]
        a_ex = a_ex.reshape((latency_params_length, 1)) @ np.ones((1, network.num_edges))
        a_gp = a_gp.reshape((latency_params_length, 1)) @ np.ones((1, network.num_edges))
    
    ## Variable indices:
    # In full:
    # y_el indices: (group, edge, "lane", time)
    # y_in indices: (group, edge, "lane", time)
    # Here:
    # y_el indices: (group, edge, "lane")
    # y_in indices: (group, edge, "lane")
    
    num_el = v_E_array.shape[0]
    num_in = v_I_array.shape[0]

    num_nodes = network.num_nodes
    num_edges = network.num_edges

#     print()
#     print("tau.shape[0]:", tau.shape[0])
#     print("num_edges:", num_edges)
#     print()

    assert tau.shape[0] == num_edges, "toll vector length must equal the number of edges."
    
    ## Compute lane flows:
    
    # Express lane (ex):
    x_ex = np.zeros(num_edges)
    for e in range(num_edges):
        x_ex[e] += sum(y_el[(g, e, k)] for g in range(num_el) for k in [0, 1])
        x_ex[e] += sum(y_in[(g, e, 0)] for g in range(num_in))
    
    # General purpose lane (gp):
    x_gp = np.zeros(num_edges)
    for e in range(num_edges):
        x_gp[e] += sum(y_el[(g, e, 2)] for g in range(num_el))
        x_gp[e] += sum(y_in[(g, e, 1)] for g in range(num_in))
        
    ell_ex = np.zeros(num_edges)
    ell_gp = np.zeros(num_edges)
    for e in range(num_edges):
        ell_ex[e] = sum(a_ex[p, e] * (x_ex[e] ** p) for p in range(5))
        ell_gp[e] = sum(a_gp[p, e] * (x_gp[e] ** p) for p in range(5))
    
    for e in range(num_edges):
        obj_E = sum(tau[e] * y_el[(g, e, 1)] for g in range(num_el)) \
                    + sum(y_el[(g, e, k)] * v_E_array[g] * ell_ex[e] for g in range(num_el) for k in [0, 1]) \
                    + sum(y_el[(g, e, 2)] * v_E_array[g] * ell_gp[e] for g in range(num_el))
        obj_I = sum(tau[e] * y_in[(g, e, 0)] for g in range(num_in)) \
                    + sum(y_in[(g, e, 0)] * v_I_array[g] * ell_ex[e] for g in range(num_in)) \
                    + sum(y_in[(g, e, 1)] * v_I_array[g] * ell_gp[e] for g in range(num_in))
        obj_R = sum(tau[e] * y_el[(g, e, 1)] for g in range(num_el)) \
                    + sum(tau[e] * y_in[(g, e, 0)] for g in range(num_in))
                    + sum((1 - alpha[e]) * tau[e] * y_el[(g, e, 1)] for g in range(num_el))
    
    welfare = lambda_E * obj_E - lambda_R * obj_R + lambda_I * obj_I
    
#     print()
#     print("obj_E:", obj_E)
#     print("obj_R:", obj_R)
#     print("obj_I:", obj_I)
#     print("welfare:", welfare)
#     print()
    
    return welfare

def latency_total(network, tau, v_I_array, v_E_array, y_el, y_in, \
                  a_ex = np.array([0.0, 0.0, 0.0, 0.0, 1.0]), \
                  a_gp = np.array([0.0, 0.0, 0.0, 0.0, 1.0])):
    
    assert np.all(a_ex >= 0.0), "All entries of a_ex must be non-negative."
    assert np.all(a_gp >= 0.0), "All entries of a_gp must be non-negative."
    assert a_ex.shape == a_gp.shape, "a_ex and a_gp must have the same shape."
    assert len(a_ex.shape) in [1, 2], "a_ex and a_gp must be either a vector or a matrix."
    
    if len(a_ex.shape) == 1:
        latency_params_length = a_ex.shape[0]
        a_ex = a_ex.reshape((latency_params_length, 1)) @ np.ones((1, network.num_edges))
        a_gp = a_gp.reshape((latency_params_length, 1)) @ np.ones((1, network.num_edges))
    
    # In full:
    # y_el indices: (group, edge, "lane", time)
    # y_in indices: (group, edge, "lane", time)
    # Here:
    # y_el indices: (group, "lane")
    # y_in indices: (group, "lane")
    
    num_el = v_E_array.shape[0]
    num_in = v_I_array.shape[0]

    num_nodes = network.num_nodes
    num_edges = network.num_edges
    
    ## Compute lane flows:
    
    # Express lane (ex):
    x_ex = np.zeros(num_edges)
    for e in range(num_edges):
        x_ex[e] += sum(y_el[(g, e, k)] for g in range(num_el) for k in [0, 1])
        x_ex[e] += sum(y_in[(g, e, 0)] for g in range(num_in))
    
    # General purpose lane (gp):
    x_gp = np.zeros(num_edges)
    for e in range(num_edges):
        x_gp[e] += sum(y_el[(g, e, 2)] for g in range(num_el))
        x_gp[e] += sum(y_in[(g, e, 1)] for g in range(num_in))
    
    # Add up latency values:
    latency_val = 0.0
    
    ell_ex = np.zeros(num_edges)
    ell_gp = np.zeros(num_edges)
    for e in range(num_edges):
        ell_ex[e] = sum(a_ex[p, e] * (x_ex[e] ** p) for p in range(5))
        ell_gp[e] = sum(a_gp[p, e] * (x_gp[e] ** p) for p in range(5))
        
        latency_val += x_ex[e] * ell_ex[e] + x_gp[e] * ell_gp[e]
    
    return latency_val



# Chinmay's Algorithm:

In [8]:
## Steps
# 1: Define variables
# 2: Define objective
# 3: Define constraints
# 4: Define problem
# 5: Solve problem
# 6: Extract values

## <font color='red'>Continue Editing Below:</font>

In [9]:

# Below: For quartic latency functions:
# Latency Function: a_4 x^4 + a_3 x^3 + a_2 x^2 + a_1 x + a_0
def solve_DBCP_direct(network, tau, alpha, v_I_array, v_E_array, \
                         a_ex = np.array([0.0, 0.0, 0.0, 0.0, 1.0]), \
                         a_gp = np.array([0.0, 0.0, 0.0, 0.0, 1.0])):
    
    assert np.all(a_ex >= 0.0), "All entries of a_ex must be non-negative."
    assert np.all(a_gp >= 0.0), "All entries of a_gp must be non-negative."
    assert a_ex.shape == a_gp.shape, "a_ex and a_gp must have the same shape."
    assert len(a_ex.shape) in [1, 2], "a_ex and a_gp must be either a vector or a matrix."
    
    if len(a_ex.shape) == 1:
        latency_params_length = a_ex.shape[0]
        a_ex = a_ex.reshape((latency_params_length, 1)) @ np.ones((1, network.num_edges))
        a_gp = a_gp.reshape((latency_params_length, 1)) @ np.ones((1, network.num_edges))
        
    ## Variable indices:
    # In full:
    # y_el indices: (group, edge, "lane", time)
    # y_in indices: (group, edge, "lane", time)
    # Here:
    # y_el indices: (group, edge, "lane")
    # y_in indices: (group, edge, "lane")
    
    num_el = v_E_array.shape[0]
    num_in = v_I_array.shape[0]

    # Variables:
    y_el = {}
    for g in range(num_el):
        for e in network.edges:
            for k in [0, 1, 2]:
                y_el[(g, e, k)] = cp.Variable(1)
            
    y_in = {}
    for g in range(num_in):
        for e in network.edges:
            for k in [0, 1]:
                y_in[(g, e, k)] = cp.Variable(1)

    x_ex = cp.Variable(network.num_edges)
    x_gp = cp.Variable(network.num_edges)
    
    # Objective:
    func = 0.0
    for e in network.edges:
        func += 1/5 * a_ex[4, e] * cp.power(x_ex[e], 5)
        func += 1/4 * a_ex[3, e] * cp.power(x_ex[e], 4)
        func += 1/3 * a_ex[2, e] * cp.power(x_ex[e], 3)
        func += 1/2 * a_ex[1, e] * cp.power(x_ex[e], 2)
        func += a_ex[0, e] * x_ex[e]
        func += 1/5 * a_gp[4, e] * cp.power(x_gp[e], 5)
        func += 1/4 * a_gp[3, e] * cp.power(x_gp[e], 4)
        func += 1/3 * a_gp[2, e] * cp.power(x_gp[e], 3)
        func += 1/2 * a_gp[1, e] * cp.power(x_gp[e], 2)
        func += a_gp[0, e] * x_gp[e]
        
        func += sum(tau[e] * y_el[(g, e, 1)] / v_E_array[g] for g in range(num_el))
        func += sum((1-alpha[e]) * tau[e] * y_el[(g, e, 0)] / v_E_array[g] for g in range(num_el))
        func += sum(tau[e] * y_in[(g, e, 0)] / v_I_array[g] for g in range(num_in))

    objective = cp.Minimize(func)
    
    # Constraints:
    constraints = []
    
    constraints += [y_el[(g, e, k)] >= 0.0 for g in range(num_el) for e in network.edges for k in [0, 1, 2]]
    constraints += [y_in[(g, e, k)] >= 0.0 for g in range(num_in) for e in network.edges for k in [0, 1]]
        
    constraints += [sum(y_el[(g, e, k)] for e in network.start_edges for k in [0, 1, 2]) == 1.0 \
                    for g in range(num_el)]
    constraints += [sum(y_in[(g, e, k)] for e in network.start_edges for k in [0, 1]) == 1.0 \
                    for g in range(num_in)]
    for node in network.intermediate_nodes:
        constraints += [sum(y_el[(g, e, k)] for e in network.nodes_dict[node]["incoming_edges"] for k in [0, 1, 2]) \
                        == sum(y_el[(g, e, k)] for e in network.nodes_dict[node]["outgoing_edges"] for k in [0, 1, 2]) \
                        for g in range(num_el)]
        constraints += [sum(y_in[(g, e, k)] for e in network.nodes_dict[node]["incoming_edges"] for k in [0, 1]) \
                        == sum(y_in[(g, e, k)] for e in network.nodes_dict[node]["outgoing_edges"] for k in [0, 1]) \
                        for g in range(num_in)]
    
    for e in network.edges:
        constraints += [x_ex[e] == sum(y_el[(g, e, k)] for g in range(num_el) for k in [0, 1]) \
                                   + sum(y_in[(g, e, 0)] for g in range(num_in)) ]
        constraints += [x_gp[e] == sum(y_el[(g, e, 2)] for g in range(num_el)) \
                                   + sum(y_in[(g, e, 1)] for g in range(num_in)) ]
    
    # Problem:
    prob = cp.Problem(objective, constraints)
    
    # Solve:
    result = prob.solve()
    
    # Extract Values:
    y_el_values = {}
    for g in range(num_el):
        for e in network.edges:
            for k in [0, 1, 2]:
                y_el_values[(g, e, k)] = max(np.round(y_el[(g, e, k)].value[0], decimals=6) , 0.0)

    y_in_values = {}
    for g in range(num_in):
        for e in network.edges:
            for k in [0, 1]:
                y_in_values[(g, e, k)] = max(np.round(y_in[(g, e, k)].value[0], decimals=6), 0.0)

    return y_el_values, y_in_values


In [ ]:
time_1 = time.time()

edges_nodes_matrix = np.array([[0, 1], [0, 2], [1, 2], [1, 3], [2, 3]])
network_Braess = network_structure(edges_nodes_matrix)
# network_Braess.print_all()

network = network_Braess

## TODO: Edit below to use the fact that creatively, CVXPY can handle multiple-dimensional n ,\
## (e.g., 3d or 4d) arrays:\n,

tau = np.array([0.2, 0.6, 0.5, 0.8, 0.4])
B = 0.8
v_I_array = np.array([1.0, 1.2, 1.4])
v_E_array = np.array([0.6, 0.8, 0.9])
flow_per_group = 1.0
a = np.array([0.0, 0.0, 0.0, 0.0, 1.0])
num_iters_max = 5000
error_bound = 1E-3
diffs_num_cols = 5
# y_init = np.array([0.0, 0.05, 0.95, 0.95, 0.05])

# lambda_E, lambda_R, lambda_I = 1.0, 1.0, 1.0
lambda_E, lambda_R, lambda_I = 10.0, 0.2, 1.0
# lambda_E, lambda_R, lambda_I = 1.0, 1.5, 1.0

# tau_max, B_max = 1.0, 1.0
flow_max = flow_per_group * (v_I_array.shape[0] + v_E_array.shape[0])
tau_max = sum(a[k] * (flow_max ** k) for k in range(5))
B_max = tau_max

d = 2
num_iters = 1000
tau = np.zeros((network.num_edges, num_iters))
tau_perturbed = np.zeros((network.num_edges, num_iters))

B = np.zeros(num_iters)
B_perturbed = np.zeros(num_iters)
delta = np.zeros(num_iters)
eta = np.zeros(num_iters)
eta_bar = 3.0
delta_bar = 3.0

welfare_list = []

# tau[:, 0] = tau_max * 0.8
# B[0] = B_max * 0.2

tau[:, 0] = tau_max * 0.7
B[0] = B_max * 0.3

# tau[:, 0] = tau_max * 0.6
# B[0] = B_max * 0.4


for i in range(num_iters-1):
    
    print()
    print("Iter:", i)
    
    eta[i] = eta_bar * (i+1)**(-1/2) * d**(-1)
    delta[i] = delta_bar * (i+1)**(-1/4) * d**(-1/2)
    w_i_unnormalized = np.random.randn(network.num_edges +1)
    w_i = w_i_unnormalized / np.linalg.norm(w_i_unnormalized)
    print("w_i:", w_i)
    tau_perturbed[:, i] = tau[:, i] + delta[i] * w_i[:-1]
    B_perturbed[i] = B[i] + delta[i] * w_i[-1]
    
#     if tau_perturbed[:, i] < B_perturbed[i] or tau_perturbed[:, i] < 0 or B_perturbed[i] < 0:
    tau_perturbed[:, i], B_perturbed[i] = proj_tau_B_11(network, tau_perturbed[:, i], B_perturbed[i], \
                                                    tau_max = tau_max, B_max = B_max)
    
    print("tau[:, i]:", tau[:, i])
    print("B[i]:", B[i])
    print("tau_perturbed[:, i]:", tau_perturbed[:, i])
    print("B_perturbed[i]:", B_perturbed[i])


    y_el_values, y_in_values = solve_DBCP_direct(network, tau = tau[:, i], B = B[i], v_I_array = v_I_array, \
                                                 v_E_array = v_E_array, a_ex = a, a_gp = a)

#     print()
#     print("y_el_values:", y_el_values)
#     print("y_in_values:", y_in_values)
#     print()

#     print()
#     print("New solve_DBCP_iter_11 call to solve_DBCP (perturbed):")
#     print()

    y_el_perturbed_values, y_in_perturbed_values = solve_DBCP_direct(network, tau = tau_perturbed[:, i], B = B_perturbed[i], \
                                                                     v_I_array = v_I_array, v_E_array = v_E_array, a_ex = a, a_gp = a)
    
#     print("y_el:", y_el)
#     print("y_in:", y_in)
#     print("y_el_perturbed:", y_el_perturbed)
#     print("y_in_perturbed:", y_in_perturbed)
    
    welfare = welfare_obj(network, lambda_E, lambda_R, lambda_I, tau = tau[:, i], v_I_array = v_I_array, v_E_array = v_E_array, \
                          y_el = y_el_values, y_in = y_in_values, alpha = alpha, a_ex = a, a_gp = a)
    welfare_perturbed = welfare_obj(network, lambda_E, lambda_R, lambda_I, tau = tau_perturbed[:, i], \
                                    v_I_array = v_I_array, v_E_array = v_E_array, \
                                    y_el = y_el_perturbed_values, y_in = y_in_perturbed_values, alpha = alpha, \
                                    a_ex = a, a_gp = a)
    welfare_list.append(welfare)
    
#     print("welfare:", welfare)
#     print("welfare_perturbed:", welfare_perturbed)
#     print("tau[:, i] - eta[i] * (d/delta[i]) * w_i[:-1] * (welfare_perturbed - welfare):\n", tau[:, i] - eta[i] * (d/delta[i]) * w_i[:-1] * (welfare_perturbed - welfare))
    
    tau[:, i+1] = tau[:, i] - eta[i] * (d/delta[i]) * w_i[:-1] * (welfare_perturbed - welfare)
    
#     print()
#     print("B[i] - eta[i] * (d/delta[i]) * w_i[-1] * (welfare_perturbed - welfare):\n", \
#          B[i] - eta[i] * (d/delta[i]) * w_i[-1] * (welfare_perturbed - welfare))
#     print("B[i]:", B[i])
#     print("eta[i]:", eta[i])
#     print("delta[i]:", delta[i])
#     print("w_i[-1]:", w_i[-1])
    print("welfare:", welfare)
    print()
    
    B[i+1] = B[i] - eta[i] * (d/delta[i]) * w_i[-1] * (welfare_perturbed - welfare)
    tau[:, i+1], B[i+1] = proj_tau_B_11(network, tau[:, i+1], B[i+1], tau_max, B_max)
    
    if i >= diffs_num_cols + 2:
        tau_diffs = np.linalg.norm(tau[:, i-diffs_num_cols : i-1] - tau[:, i-diffs_num_cols+1 : i], axis = 0)
        B_diffs = B[i-diffs_num_cols : i-1] - B[i-diffs_num_cols+1 : i]
        
#         print("tau[:, 0:10]:", tau[:, 0:10])
#         print("B[0:10]:", B[0:10])
        print("tau_diffs:", tau_diffs)
        print("B_diffs:", B_diffs)
        
        if max(np.max(np.absolute(tau_diffs)), np.max(np.absolute(B_diffs))) < error_bound:
            break

time_2 = time.time()

min_welfare = min(welfare_list)
argmin_welfare_list = welfare_list.index(min(welfare_list))
argmin_tau = tau[:, argmin_welfare_list]
argmin_B = B[argmin_welfare_list]

print()
print("Time:", time_2 - time_1)



In [ ]:
print("first(welfare_list):", welfare_list[0])
print("min(welfare_list):", min(welfare_list))
print("max(welfare_list):", max(welfare_list))
print("argmin_tau:", argmin_tau)
print("argmin_B:", argmin_B)

In [ ]:
y_el_values

In [ ]:
y_in_values

## Test:

In [ ]:
# Test:

grad = np.array([3.11430535, 1.501, 1.501, 2.46858321, 1.501])

# y_el: \hat y_1 E, \tilde y_1 E, y_2 E
y_el_var = cp.Variable(3)
# y_in: y_1 I, y_2 I
y_in_var = cp.Variable(2)

objective = cp.Minimize(grad[0:3] @ y_el_var + grad[3:] @ y_in_var)

constraints = []
constraints += [y_el_var >= 0, y_in_var >= 0]
constraints += [cp.sum(y_el_var) == 1.0, cp.sum(y_in_var) == 1.0]
constraints += [y_el_var[1] * tau <= B]

prob = cp.Problem(objective, constraints)
result = prob.solve()

print("grad:", grad)
print("y_el_var.value:", y_el_var.value)
# print("y_el_var_current:", y_el_var_current)
print("y_in_var.value:", y_in_var.value)
# print("y_in_var_current:", y_in_var_current)
print()

# y_el_var_current = y_el_var_current + 2/(k+2) * (y_el_var.value - y_el_var_current)
# y_in_var_current = y_in_var_current + 2/(k+2) * (y_in_var.value - y_in_var_current)

# y_iters[0:3, k] = y_el_var_current
# y_iters[3:, k] = y_in_var_current

# The optimal objective value is returned by `prob.solve()`.
result = prob.solve()

# Solver=SCS,verbose=False

In [ ]:
# y_el_var
# y_in_var
# np.hstack((y_el_var, y_in_var))

# Unused Functions for Network:

In [ ]:

def construct_depth_dicts(self):
    self.depth_edges_dict = {}
    counter = 1

    edges_remaining = list(self.edges)
    edges_accounted = []
#         print("[edge for edge in edges_remaining]:", [edge for edge in edges_remaining])
    edges_temp = [edge for edge in edges_remaining if len(self.edges_dict[edge]['incoming_edges']) == 0]
    self.depth_edges_dict[1] = edges_temp
    edges_remaining = [edge for edge in edges_remaining if edge not in edges_temp]
    edges_accounted.extend(edges_temp)
    counter += 1

    while edges_remaining != []:
#             print("[edge for edge in edges_remaining]:", [edge for edge in edges_remaining])
        edges_temp = [edge for edge in edges_remaining if set(self.edges_dict[edge]['incoming_edges']).issubset(set(edges_accounted))]
        self.depth_edges_dict[counter] = edges_temp
        edges_remaining = [edge for edge in edges_remaining if edge not in edges_temp]
        edges_accounted.extend(edges_temp)
        counter += 1

    self.depths = list(self.depth_edges_dict.keys())

    self.depth_nodes_dict = {}
    for depth in self.depths:
        nodes_temp = np.array([self.edges_dict[edge]['start_node'] for edge in self.depth_edges_dict[depth]])
        nodes_temp = list(np.unique(nodes_temp.flatten()))
        self.depth_nodes_dict[depth] = nodes_temp

    return

def construct_height_dicts(self):
    self.height_edges_dict = {}
    counter = 1

    edges_remaining = list(self.edges)
    edges_accounted = []
#         print("[edge for edge in edges_remaining]:", [edge for edge in edges_remaining])
    edges_temp = [edge for edge in edges_remaining if len(self.edges_dict[edge]['outgoing_edges']) == 0]
    self.height_edges_dict[1] = edges_temp
    edges_remaining = [edge for edge in edges_remaining if edge not in edges_temp]
    edges_accounted.extend(edges_temp)
    counter += 1

    while edges_remaining != []:
#             print("[edge for edge in edges_remaining]:", [edge for edge in edges_remaining])
        edges_temp = [edge for edge in edges_remaining if set(self.edges_dict[edge]['outgoing_edges']).issubset(set(edges_accounted))]
        self.height_edges_dict[counter] = edges_temp
        edges_remaining = [edge for edge in edges_remaining if edge not in edges_temp]
        edges_accounted.extend(edges_temp)
        counter += 1

    self.heights = list(self.height_edges_dict.keys())

    self.height_nodes_dict = {}
    for height in self.heights:
        nodes_temp = np.array([self.edges_dict[edge]['end_node'] for edge in self.height_edges_dict[height]])
        nodes_temp = list(np.unique(nodes_temp.flatten()))
        self.height_nodes_dict[height] = nodes_temp

    return

# Scratch Work:

In [ ]:
x = cp.Variable(2)
y = cp.Variable(2)
v_fixed = np.array([0, 1])
objective = cp.Minimize(cp.sum_squares(x - y) + cp.sum_squares(x - v_fixed))
constraints = []
prob = cp.Problem(objective, constraints)

# The optimal objective value is returned by `prob.solve()`.
result = prob.solve()
# The optimal value for x is stored in `x.value`.
print("x.value:", x.value)
print("y.value:", y.value)
print()


In [ ]:
# # Test:

# Variables:

# y_el: \hat y_1 E, \tilde y_1 E, y_2 E
y_elig = cp.Variable(3)
# y_in: y_1 I, y_2 I
y_inel = cp.Variable(2)


# Objective:

a = np.array([0.0, 0.0, 0.0, 0.0, 1.0])

tau = 0.5
B = 0.4

func = 0.0
func += 1/5 * a[4] * cp.power(y_elig[0] + y_elig[1] + y_inel[0], 5)
func += 1/4 * a[3] * cp.power(y_elig[0] + y_elig[1] + y_inel[0], 4)
func += 1/3 * a[2] * cp.power(y_elig[0] + y_elig[1] + y_inel[0], 3)
func += 1/2 * a[1] * cp.power(y_elig[0] + y_elig[1] + y_inel[0], 2)
func += a[0] * (y_elig[0] + y_elig[1] + y_inel[0])
func += y_inel[0] * tau / v_I + y_elig[0] * tau / v_E
func += 1/5 * a[4] * cp.power(y_elig[2] + y_inel[1], 5)
func += 1/4 * a[3] * cp.power(y_elig[2] + y_inel[1], 4)
func += 1/3 * a[2] * cp.power(y_elig[2] + y_inel[1], 3)
func += 1/2 * a[1] * cp.power(y_elig[2] + y_inel[1], 2)
func += a[0] * (y_elig[2] + y_inel[1])

objective = cp.Minimize(func)


# Constraints:

constraints = []
constraints += [y_elig >= 0, y_inel >= 0]
constraints += [cp.sum(y_elig) == 1, cp.sum(y_inel) == 1]
constraints += [y_elig[1] * tau <= B]

# Solve problem:
prob = cp.Problem(objective, constraints)
result = prob.solve()

# Print solution:
print("y_elig.value:", np.round(y_elig.value, 4) )
print("y_inel.value:", np.round(y_inel.value, 4) )
print()



# power(x, p)



In [ ]:
x1 = 1
x2 = 2
print("x1:", x1, ", x2:", x2)

## CVXPY can handle 4d arrays:

In [ ]:

I, J, K, L = 2, 3, 4, 5

# Variables:
x_test = {}
for i in range(I):
    for j in range(J):
        for k in range(K):
            for ell in range(L):
                x_test[(i, j, k, ell)] = cp.Variable(1)
            
# Objective:

func = 0.0
func += cp.sum([x_test[(i, j, k, ell)]**2 for i in range(I) for j in range(J) \
                for k in range(K) for ell in range(L)])
            
objective = cp.Minimize(func)

# Constraints:
constraints = []

for i in range(I):
    for j in range(J):
        for k in range(K):
            for ell in range(L):
                constraints += [cp.sum([x_test[(i, j, k, ell)] for i in range(I) for j in range(J) \
                                        for k in range(K) for ell in range(L) ]) == 1.0]
                constraints += [x_test[(i, j, k, ell)] >= 0.0 for i in range(I) for j in range(J) \
                                        for k in range(K) for ell in range(L)]

# Solve problem:
prob = cp.Problem(objective, constraints)
result = prob.solve()

# Print solution:
for i in range(I):
    for j in range(J):
        for k in range(K):
            for ell in range(L):
                print("i, j, k, ell:", i, j, k, ell)
                print("x_test[(i,j,k, ell)].value:", x_test[(i, j, k, ell)].value)
